In [34]:
import pandas as pd
import re
from difflib import SequenceMatcher

# Load the datasets
left_dataset_path = '/Users/martinng/Desktop/Columbia/2024_Spring/Python for Data Analysis/Project/Data/GroupProject/entity-resolution/left_dataset.csv'
right_dataset_path = '/Users/martinng/Desktop/Columbia/2024_Spring/Python for Data Analysis/Project/Data/GroupProject/entity-resolution/right_dataset.csv'

left_df = pd.read_csv(left_dataset_path)
right_df = pd.read_csv(right_dataset_path)

In [35]:
def clean_text(text):
    """Lowercase, remove special characters, and strip whitespace."""
    text = text.lower()
    text = re.sub(r'[\W_]+', ' ', text)  # Replace all non-word characters with space
    return text.strip()

# Apply text cleaning
left_df['name'] = left_df['name'].apply(clean_text)
left_df['state'] = left_df['state'].apply(clean_text)
left_df['city'] = left_df['city'].apply(clean_text)
right_df['name'] = right_df['name'].apply(clean_text)
right_df['state'] = right_df['state'].apply(clean_text)
right_df['city'] = right_df['city'].apply(clean_text)

In [36]:
def standardize_zip_code(zip_code):
    """Ensure zip code is a string, truncate or pad to 5 digits."""
    zip_code = str(zip_code)
    return zip_code[:5].zfill(5)  # Pad or truncate to ensure 5 characters

left_df['postal_code'] = left_df['postal_code'].apply(standardize_zip_code)
right_df['zip_code'] = right_df['zip_code'].apply(standardize_zip_code)

In [37]:
def clean_address(address):
    """Standardize and clean the address, handling NaN values gracefully."""
    if pd.isna(address):
        return ""
    address = address.lower()
    address = re.sub(r'\bstreet\b', 'st', address)
    address = re.sub(r'\broad\b', 'rd', address)
    address = re.sub(r'\bavenue\b', 'ave', address)
    address = re.sub(r'\bdrive\b', 'dr', address)
    address = re.sub(r'[^a-zA-Z0-9\s]', '', address)
    address = re.sub(r'\s+', ' ', address).strip()
    return address

left_df['address'] = left_df['address'].apply(clean_address)
right_df['address'] = right_df['address'].apply(clean_address)

In [38]:
# Extra Step to drop unwanted columns and standardize column names
left_df.drop(columns=['categories'], inplace=True)
right_df.drop(columns=['size'], inplace=True)

left_df.rename(columns={'postal_code': 'zip_code'}, inplace=True)

In [39]:
def create_enhanced_block_keys(df):
    """Generate block keys by combining several fields."""
    df['block_key'] = df.apply(lambda x:
                                (x['name'][0].lower() if pd.notna(x['name']) and x['name'] != "" else '0') +
                                "_" + (x['state'].upper() if pd.notna(x['state']) else 'Unknown') +
                                "_" + (x['city'][0].lower() if pd.notna(x['city']) and x['city'] != "" else '0') +
                                "_" + (str(x['zip_code'])[:5] if pd.notna(x['zip_code']) else '00000'), axis=1)
    return df

In [40]:
left_df = create_enhanced_block_keys(left_df)
right_df = create_enhanced_block_keys(right_df)

left_df['combined'] = left_df['name'].apply(clean_text) + " " + left_df['address'].apply(clean_address)
right_df['combined'] = right_df['name'].apply(clean_text) + " " + right_df['address'].apply(clean_address)

In [41]:
def fuzzy_match_with_difflib(left_df, right_df, threshold=0.80):
    results = []
    common_blocks = set(left_df['block_key']).intersection(set(right_df['block_key']))

    for block in common_blocks:
        left_block = left_df[left_df['block_key'] == block]
        right_block = right_df[right_df['block_key'] == block]
        for _, left_row in left_block.iterrows():
            best_match = None
            best_score = threshold
            for _, right_row in right_block.iterrows():
                score = SequenceMatcher(None, left_row['combined'], right_row['combined']).ratio()
                if score > best_score:
                    best_score = score
                    best_match = right_row['business_id']

            if best_match:
                match_data = {
                    'left_id': left_row['entity_id'],
                    'right_id': best_match,
                    'match_score': best_score
                }
                results.append(match_data)

    return pd.DataFrame(results)

In [44]:
matched_results = fuzzy_match_with_difflib(left_df, right_df,0.6)
print(matched_results)

       left_id  right_id  match_score
0         6355     54035     0.883117
1        32241     75157     0.935484
2        49462     54385     0.615385
3        54382     71170     0.961538
4        74713     54860     0.738095
...        ...       ...          ...
22951    41738     40255     0.842105
22952    59208     41959     0.781250
22953    25560     23897     0.935484
22954    23247     76128     0.941176
22955    54529     57701     0.952381

[22956 rows x 3 columns]
